# 1 — Installing ROS2 & Essential Tools

> Step-by-step guide to install ROS2, colcon, and the packages you'll need for this course.

**Goal:** By the end of this notebook you will have a fully working ROS2 environment and understand *why* each piece is installed.

In [ ]:
#| default_exp installation

## 1.1 Platform Decision: Ubuntu on WSL2

Since you're on **Windows**, the best way to run ROS2 is through **WSL2** (Windows Subsystem for Linux). WSL2 gives you a real Linux kernel with near-native performance.

### Why WSL2?
- ROS2 has **first-class support** on Ubuntu
- Most ROS2 tutorials, packages, and community answers target Ubuntu  
- USB device passthrough works (for cameras!)
- GUI apps (RViz2, Gazebo) work via WSLg

### Install WSL2 + Ubuntu

Open **PowerShell as Administrator** and run:

```powershell
# Install WSL2 with Ubuntu 22.04 (for Humble) or 24.04 (for Jazzy)
wsl --install -d Ubuntu-22.04

# Or for the latest:
# wsl --install -d Ubuntu-24.04
```

After installation, launch Ubuntu from the Start Menu and create your username/password.

> **Important:** All commands from this point forward run **inside WSL2** (the Ubuntu terminal), not in PowerShell.

## 1.2 Choosing a ROS2 Distribution

| Distribution | Ubuntu | Why choose it |
|-------------|--------|---------------|
| **Humble** | 22.04 | Most tutorials target this; widest package support; LTS until 2027 |
| **Jazzy** | 24.04 | Newest LTS; modern packages; LTS until 2029 |

We'll show commands for **both**. Pick one and stick with it.

```bash
# Check your Ubuntu version
lsb_release -a
```

## 1.3 Install ROS2 — Full Step-by-Step

### Step 1: Set Locale (UTF-8)

ROS2 requires UTF-8 locale. Most Ubuntu installs already have it, but let's make sure:

```bash
sudo apt update && sudo apt install -y locales
sudo locale-gen en_US en_US.UTF-8
sudo update-locale LC_ALL=en_US.UTF-8 LANG=en_US.UTF-8
export LANG=en_US.UTF-8
```

### Step 2: Enable the ROS2 Repository

```bash
# Ensure Ubuntu Universe repository is enabled
sudo apt install -y software-properties-common
sudo add-apt-repository universe

# Add the ROS2 GPG key
sudo apt update && sudo apt install -y curl
sudo curl -sSL https://raw.githubusercontent.com/ros/rosdistro/master/ros.key \
  -o /usr/share/keyrings/ros-archive-keyring.gpg

# Add the repository to your sources list
echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/ros-archive-keyring.gpg] \
  http://packages.ros.org/ros2/ubuntu $(. /etc/os-release && echo $UBUNTU_CODENAME) main" \
  | sudo tee /etc/apt/sources.list.d/ros2.list > /dev/null
```

### Step 3: Install ROS2

```bash
sudo apt update
sudo apt upgrade -y

# === For Humble (Ubuntu 22.04) ===
sudo apt install -y ros-humble-desktop

# === For Jazzy (Ubuntu 24.04) ===
# sudo apt install -y ros-jazzy-desktop
```

#### What does `ros-<distro>-desktop` include?

| Component | What it is |
|-----------|------------|
| `ros-base` | Core libraries: `rclpy`, `rclcpp`, message types, launch system |
| `rviz2` | 3D visualization tool |
| `rqt` | Qt-based GUI tools (topic monitor, graph viewer) |
| `demo_nodes_*` | Example publisher/subscriber nodes for testing |
| `rosbag2` | Record and replay topic data |

> **Why `desktop` and not `ros-base`?** Because `desktop` includes RViz2 and rqt which are essential for debugging. The extra ~500 MB is worth it.

## 1.4 Source the ROS2 Setup Script

ROS2 installs its binaries and libraries in `/opt/ros/<distro>/`. To use them, you must **source** the setup script:

```bash
# For Humble:
source /opt/ros/humble/setup.bash

# For Jazzy:
# source /opt/ros/jazzy/setup.bash
```

### Make it permanent (add to ~/.bashrc)

```bash
# Add to the END of your ~/.bashrc so it loads on every terminal:
echo 'source /opt/ros/humble/setup.bash' >> ~/.bashrc
```

> **What does sourcing do?** It sets environment variables (`ROS_DISTRO`, `AMENT_PREFIX_PATH`, `PYTHONPATH`, etc.) so your shell knows where ROS2 libraries and executables live.

## 1.5 Verify the Installation

```bash
# Check ROS2 is working
ros2 --help

# Check the distribution
echo $ROS_DISTRO
# Should print: humble  (or jazzy)

# Run the demo talker in one terminal:
ros2 run demo_nodes_cpp talker

# In ANOTHER terminal, run the listener:
ros2 run demo_nodes_cpp listener
```

You should see the talker printing `Publishing: 'Hello World: 1'` and the listener printing `I heard: [Hello World: 1]`. **Congratulations — ROS2 is working!**

Press `Ctrl+C` to stop both.

## 1.6 Install the Build Tools

ROS2 uses **colcon** to build packages and **rosdep** to install dependencies:

```bash
# colcon — the build tool
sudo apt install -y python3-colcon-common-extensions

# rosdep — automatic dependency installer
sudo apt install -y python3-rosdep
sudo rosdep init     # Only needed once
rosdep update
```

### Why colcon?

- Builds **multiple packages** in dependency order
- Supports **both C++ and Python** packages in one workspace
- `colcon build` → compiles everything
- `colcon test` → runs all tests

### Why rosdep?

When you add a community package to your workspace, it may need system libraries (e.g., OpenCV, Eigen). `rosdep install` reads the package's declared dependencies and installs them automatically.

## 1.7 Install Essential Extra Packages

These are packages we'll use throughout this course:

```bash
# Replace 'humble' with 'jazzy' if you're on Jazzy
DISTRO=humble

sudo apt install -y \
  ros-${DISTRO}-usb-cam \
  ros-${DISTRO}-v4l2-camera \
  ros-${DISTRO}-image-transport \
  ros-${DISTRO}-cv-bridge \
  ros-${DISTRO}-vision-opencv \
  ros-${DISTRO}-image-pipeline \
  ros-${DISTRO}-compressed-image-transport \
  ros-${DISTRO}-camera-info-manager \
  python3-opencv
```

### What each package does:

| Package | Purpose | Used in notebook |
|---------|---------|------------------|
| `usb-cam` | USB webcam driver node — publishes camera images as ROS topics | 06 |
| `v4l2-camera` | Alternative camera driver using Video4Linux2 | 06 |
| `image-transport` | Efficient image topic transport (raw, compressed, theora) | 06, 07 |
| `cv-bridge` | Converts between ROS `Image` messages and OpenCV `numpy` arrays | 07, 08 |
| `vision-opencv` | Meta-package pulling in cv_bridge + image_geometry | 07, 08 |
| `image-pipeline` | Rectification, color conversion, stereo processing | 07 |
| `compressed-image-transport` | Send compressed images (JPEG/PNG) over topics — saves bandwidth | 06 |
| `camera-info-manager` | Manages camera calibration data | 06 |
| `python3-opencv` | OpenCV Python bindings (system-level) | 07, 08 |

## 1.8 Install Python Packages (for Deep Learning)

For the deep learning notebook (08), we'll need some Python packages. Install them in your **uv/venv** or system-wide:

```bash
# Using pip (inside your ROS2 environment)
pip3 install ultralytics opencv-python-headless numpy

# ultralytics gives us YOLOv8/v11 out of the box
# opencv-python-headless avoids conflicts with the system OpenCV
```

> **Note:** In ROS2 Python nodes, you import system-installed packages. If you use a virtualenv, make sure to include `--system-site-packages` so ROS2 libraries (`rclpy`, `sensor_msgs`, etc.) are accessible.

## 1.9 USB Camera Passthrough (WSL2 Only)

If you're on WSL2 and want to use a physical USB camera, you need **usbipd-win**:

### On Windows (PowerShell as Admin):

```powershell
# Install usbipd
winget install usbipd

# List USB devices
usbipd list

# Bind the camera (note the BUSID, e.g., 1-3)
usbipd bind --busid <BUSID>

# Attach to WSL
usbipd attach --wsl --busid <BUSID>
```

### Inside WSL2:

```bash
# Verify the camera appears
ls /dev/video*
# Should show /dev/video0 (and possibly video1)

# Install v4l-utils to inspect the camera
sudo apt install -y v4l-utils
v4l2-ctl --list-devices
```

## 1.10 Environment Setup Script

Here's a complete `~/.bashrc` addition that sets up your ROS2 environment on every terminal launch:

In [ ]:
#| export

BASHRC_ADDITIONS = """
# ──────────────────────────────────────────────
# ROS2 Environment Setup
# ──────────────────────────────────────────────

# Source the ROS2 installation
source /opt/ros/humble/setup.bash      # Change to 'jazzy' if needed

# Source your workspace (after you've built it — see notebook 03)
# source ~/ros2_ws/install/setup.bash

# Set the default DDS implementation (CycloneDDS is lightweight and reliable)
export RMW_IMPLEMENTATION=rmw_cyclonedds_cpp

# Set ROS_DOMAIN_ID so your traffic doesn't clash with classmates/colleagues
# Pick any number 0-232 unique to you
export ROS_DOMAIN_ID=42

# Enable colcon autocomplete
source /usr/share/colcon_argcomplete/hook/colcon-argcomplete.bash

# Handy aliases
alias cb='cd ~/ros2_ws && colcon build --symlink-install'
alias cs='source ~/ros2_ws/install/setup.bash'
alias cbs='cb && cs'
"""

print(BASHRC_ADDITIONS)


# ──────────────────────────────────────────────
# ROS2 Environment Setup
# ──────────────────────────────────────────────

# Source the ROS2 installation
source /opt/ros/humble/setup.bash      # Change to 'jazzy' if needed

# Source your workspace (after you've built it — see notebook 03)
# source ~/ros2_ws/install/setup.bash

# Set the default DDS implementation (CycloneDDS is lightweight and reliable)
export RMW_IMPLEMENTATION=rmw_cyclonedds_cpp

# Set ROS_DOMAIN_ID so your traffic doesn't clash with classmates/colleagues
# Pick any number 0-232 unique to you
export ROS_DOMAIN_ID=42

# Enable colcon autocomplete
source /usr/share/colcon_argcomplete/hook/colcon-argcomplete.bash

# Handy aliases
alias cb='cd ~/ros2_ws && colcon build --symlink-install'
alias cs='source ~/ros2_ws/install/setup.bash'
alias cbs='cb && cs'



### Why each setting matters:

| Setting | Purpose |
|---------|--------|
| `source /opt/ros/.../setup.bash` | Makes all ROS2 commands available |
| `RMW_IMPLEMENTATION` | Explicitly selects the DDS vendor — avoids surprises |
| `ROS_DOMAIN_ID` | Isolates your network traffic (important on shared WiFi / labs) |
| `colcon-argcomplete` | Tab completion for `colcon build`, `colcon test`, etc. |
| `cb`, `cs`, `cbs` | Shortcuts: **c**olcon **b**uild, **c**olcon **s**ource, build+source |

## 1.11 Summary & Checklist

After this notebook you should have:

- [x] WSL2 + Ubuntu installed (if on Windows)
- [x] ROS2 Humble (or Jazzy) installed via `ros-<distro>-desktop`
- [x] `colcon` and `rosdep` installed
- [x] Camera, OpenCV, and image transport packages installed
- [x] `~/.bashrc` properly sourcing ROS2
- [x] `ros2 run demo_nodes_cpp talker` / `listener` working
- [x] (Optional) USB camera passing through to WSL2

---

**Next →** [Notebook 02: ROS2 Architecture & Core Concepts](02_ros2_architecture.ipynb)